# Azml II datasets - trainings
Laurent Cetinsoy

Dans ce notebook on continue notre exploration d'AZML avec les Datasets, les entraînements avec AZML, la notion de compute target, d'inference config et de déploiement


## Datasets

Azml permet de gérer ses propres dataset avec la class... Tada... Dataset. 
La classe comporte plusieurs classes filles. Quelles sont elles?




TabularDataset et FileDataset elles sont dans le module azureml.data.Tabulare pour TabularDataset

Remarque il existe aussi la classe azureml.core.Dataset

**Attention** ne pas confondre `Datastore` et `Dataset`

Quelles sont les méthodes importantes de ces différentes classes d'après vous ? 

- register : qui permet d'ajouter au workspace un dataset qu'on a créé à partir du datastore
- random_split : fais un split du dataset


Pour les tabulars : 
cf : https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabular_dataset.tabulardataset?view=azure-ml-py

- drop_column
- take_sample


Charger le workspace créé lors du précédent notebook.

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config("config.json")
ws.get_details()

{'id': '/subscriptions/50f55159-0c1b-4154-9452-e079e6b5e663/resourceGroups/AZML/providers/Microsoft.MachineLearningServices/workspaces/AZML1',
 'name': 'AZML1',
 'location': 'francecentral',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '72323e56-d933-4fb7-baf3-4df99afb556f',
 'description': '',
 'friendlyName': 'AZML1',
 'creationTime': '2020-07-23T07:04:20.9492237+00:00',
 'keyVault': '/subscriptions/50f55159-0c1b-4154-9452-e079e6b5e663/resourcegroups/azml/providers/microsoft.keyvault/vaults/azml11896339130',
 'applicationInsights': '/subscriptions/50f55159-0c1b-4154-9452-e079e6b5e663/resourcegroups/azml/providers/microsoft.insights/components/azml18112192977',
 'identityPrincipalId': '301204a0-b322-462b-927e-95ffc4a92070',
 'identityTenantId': '1d719a42-0e65-4ed7-ad1a-9434ee408dec',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/50f55159-0c1b-4154-9452-e079e6b5e663/resourcegroups/azml/providers/microsoft

A l'aide du bon Datastore, uploader le fichier dataset1 dans le datastore

In [2]:
ds = ws.get_default_datastore()
type(ds)# c'est un blob storage

azureml.data.azure_storage_datastore.AzureBlobDatastore

Uploader le fichier dataset1.csv à l'aide du datastore

In [3]:
ds.upload_files(['./dataset1.csv'])

Uploading an estimated of 1 files
Target already exists. Skipping upload for dataset1.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_workspaceblobstore

Récupérer une référence vers ce dataset avec la classe Dataset.Tabular 

In [4]:
from azureml.core import Dataset
source = [(ds, 'dataset1.csv')]
dataset = Dataset.Tabular.from_delimited_files(path=source)
type(dataset)

azureml.data.tabular_dataset.TabularDataset

Afficher avec le sdk la liste de tous les datasets disponibles de votre workspace

In [5]:
from azureml.data.tabular_dataset import TabularDataset
TabularDataset.get_all(ws)

{'CountryCode_droped': DatasetRegistration(id='a7c3b231-9a4c-4193-b19d-026d0bac1ab6', name='CountryCode_droped', version=1, description='', tags={}), 'dataset_superbe': DatasetRegistration(id='2e28ae6b-04a5-451a-a76f-0fe9f1ee01db', name='dataset_superbe', version=1, description='', tags={}), 'superbedatasetuploaded': DatasetRegistration(id='b2210b06-6e7c-4d82-8311-4befa4cd34ec', name='superbedatasetuploaded', version=1, description='', tags={})}

On voit que le dataset n'est pas présent, en effet il faut le register nous même. Register le dataset

In [6]:
dataset.register(ws, "dataset_super")

{
  "source": [
    "('workspaceblobstore', 'dataset1.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "58f921a2-556b-4e73-9435-ce204e13a2bd",
    "name": "dataset_super",
    "version": 1,
    "workspace": "Workspace.create(name='AZML1', subscription_id='50f55159-0c1b-4154-9452-e079e6b5e663', resource_group='AZML')"
  }
}

Afficher à nouveau le dataset et vérifier sur le portail aussi

In [7]:
dataset.to_pandas_dataframe()

/home/elka/anaconda3/envs/env_azml/lib/python3.7/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.12.0 --upgrade
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


,Country Name,Country Code,Birth rate,Internet users,Income Group
0,Aruba,ABW,10.244,78.90000,High income
1,Afghanistan,AFG,35.253,5.90000,Low income
2,Angola,AGO,45.985,19.10000,Upper middle income
3,Albania,ALB,12.877,57.20000,Upper middle income
4,United Arab Emirates,ARE,11.044,88.00000,High income
5,Argentina,ARG,17.716,59.90000,High income
6,Armenia,ARM,13.308,41.90000,Lower middle income
7,Antigua and Barbuda,ATG,16.447,63.40000,High income
8,Australia,AUS,13.200,83.00000,High income
9,Austria,AUT,9.400,80.61880,High income


In [8]:
TabularDataset.get_all(ws)

{'dataset_super': DatasetRegistration(id='58f921a2-556b-4e73-9435-ce204e13a2bd', name='dataset_super', version=1, description='', tags={}), 'CountryCode_droped': DatasetRegistration(id='a7c3b231-9a4c-4193-b19d-026d0bac1ab6', name='CountryCode_droped', version=1, description='', tags={}), 'dataset_superbe': DatasetRegistration(id='2e28ae6b-04a5-451a-a76f-0fe9f1ee01db', name='dataset_superbe', version=1, description='', tags={}), 'superbedatasetuploaded': DatasetRegistration(id='b2210b06-6e7c-4d82-8311-4befa4cd34ec', name='superbedatasetuploaded', version=1, description='', tags={})}

Supprimer la première colonne du dataset et afficher le dataset avec la méthode to_pandas_dataframe

In [9]:
droped_dataset = dataset.drop_columns("Country Code")
droped_dataset.to_pandas_dataframe()

## Méthode 2 utile pour ne récupérer que la colonne de label
#income_col = dataset.keep_columns('Income Group')
#income_col.to_pandas_dataframe()

,Country Name,Birth rate,Internet users,Income Group
0,Aruba,10.244,78.90000,High income
1,Afghanistan,35.253,5.90000,Low income
2,Angola,45.985,19.10000,Upper middle income
3,Albania,12.877,57.20000,Upper middle income
4,United Arab Emirates,11.044,88.00000,High income
5,Argentina,17.716,59.90000,High income
6,Armenia,13.308,41.90000,Lower middle income
7,Antigua and Barbuda,16.447,63.40000,High income
8,Australia,13.200,83.00000,High income
9,Austria,9.400,80.61880,High income


In [10]:
type(droped_dataset)

azureml.data.tabular_dataset.TabularDataset

Reuploader le dataset en changeant de nom l'upload

In [11]:
droped_dataset.register(ws, "CountryCode_droped")
TabularDataset.get_all(ws)

{'dataset_super': DatasetRegistration(id='58f921a2-556b-4e73-9435-ce204e13a2bd', name='dataset_super', version=1, description='', tags={}), 'CountryCode_droped': DatasetRegistration(id='a7c3b231-9a4c-4193-b19d-026d0bac1ab6', name='CountryCode_droped', version=1, description='', tags={}), 'dataset_superbe': DatasetRegistration(id='2e28ae6b-04a5-451a-a76f-0fe9f1ee01db', name='dataset_superbe', version=1, description='', tags={}), 'superbedatasetuploaded': DatasetRegistration(id='b2210b06-6e7c-4d82-8311-4befa4cd34ec', name='superbedatasetuploaded', version=1, description='', tags={})}

Faire un split du dataset

In [12]:
sub_data1, sub_data2 = dataset.random_split(0.8)
type(sub_data1), type(sub_data2)

(azureml.data.tabular_dataset.TabularDataset,
 azureml.data.tabular_dataset.TabularDataset)

In [13]:
sub_data1.to_pandas_dataframe().shape, sub_data2.to_pandas_dataframe().shape

/home/elka/anaconda3/envs/env_azml/lib/python3.7/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.12.0 --upgrade
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


((156, 5), (39, 5))

Uploader puis charger le dataset2.csv

In [14]:
ds.upload_files(['./dataset2.csv'])

Uploading an estimated of 1 files
Target already exists. Skipping upload for dataset2.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_workspaceblobstore

Est-il possible de concatener les deux datasets horizontalement ? 

oui

 Si c'est possible, concatener les deux dataset au sein d'un même objet dataset 

In [15]:
ds_paths = [
    (ds, 'dataset1.csv'),
    (ds, 'dataset2.csv')
           ]
dataset_concat = Dataset.Tabular.from_delimited_files(path=ds_paths)
dataset_concat.to_pandas_dataframe()

,Country Name,Country Code,Birth rate,Internet users,Income Group
0,Aruba,ABW,10.244,78.90000,High income
1,Afghanistan,AFG,35.253,5.90000,Low income
2,Angola,AGO,45.985,19.10000,Upper middle income
3,Albania,ALB,12.877,57.20000,Upper middle income
4,United Arab Emirates,ARE,11.044,88.00000,High income
5,Argentina,ARG,17.716,59.90000,High income
6,Armenia,ARM,13.308,41.90000,Lower middle income
7,Antigua and Barbuda,ATG,16.447,63.40000,High income
8,Australia,AUS,13.200,83.00000,High income
9,Austria,AUT,9.400,80.61880,High income


Est-il possible faire un join comme en pandas avec la classe Dataset ? 

A priori ce n'est pas possible mais peut être que je me trompe. Pour moi la classe dataset n'a **pas** vocation a remplacer pandas mais à simplement manipuler les dataset sur Azure machine learning 
Néanmoins, elle  propose quelques méthodes qui peuvent être utiles dans le cas du ML (split, sample, etc) mais ça reste basique

## Entraînement version II

Dans le notebook précédent, on a entraîné notre modèle nous même et on a ajouté des run.log. 

Azml propose aussi des outils pour gérer l'entraînement des modèles. L'idée est d'avoir son fichier d'entraînement sur son pc et de l'envoyer sur l'environnement d'entraînement de votre choix (une ressource azure bien entendu) ou en local.

Pour cela on va avoir besoin notamment classes ComputeTarget et Estimator.



Aller sur la page de la classe Estimator, à quoi sert-elle ? 

### Entraînement local

Créer une experiment rattaché à votre workspace

In [16]:
from azureml.core import Experiment
exp = Experiment(ws, "train_estimator")

Reprendre la petite fonction d'hier d'entraînement du modèle et la mettre dans un fichier "train.py".
Récupérer une variable run au début du script et faire en sorte que l'entraînement soit lancé chaque fois qu'on lance le script

Créer une instance de SkLearnEstimator qui utilise le fichier train.py

Références utiles :
- https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py

- https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/training/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb

In [20]:
from azureml.train.sklearn import SKLearn
estimator = SKLearn(source_directory=".", 
                    compute_target="local",
                    entry_script='train.py'
                   )

Lancer l'entraînement de votre estimator avec la méthode experiment.submit. Utiliser l'argument show_output = True

In [21]:
exp.submit(estimator)

Experiment,Id,Type,Status,Details Page,Docs Page
train_estimator,train_estimator_1595582934_9c671ac4,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


Vérifier que ça marche : l'entraînement est bien lancé et que les résultats sont bien visibles sur le dashboard.

Modifier le script pour qu'après chaque entraînement le modèle soit sauvegardé dans un sous dossier output

### Entraînement sur environement distant 

On va adapter notre entraînement pour que ça marche à distance. 

Créer une Azml Compute Instance  grâce à la méthode provisioning_configuration de la classe AmlCompute. Il va retourne un objet de type ComputeConfig

**IL FAUDRA SUPPRIMER LA COMPUTE INSTANCE EN ALLANT DANS VOTRE WORKSPACE DANS LE PORTAIL ET CLIQUE SUR COMPUTE INSTANCE ET COMPUTE CLUSTER**

In [ ]:
from azureml.core.compute import AmlCompute
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

Grâce à l'objet, créer une ComputeTarget qui référence le AmlCompute qui vient d'être créé

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
cpu_cluster = ComputeTarget.create(ws, "elkalkul", compute_config)

In [22]:
from azureml.core.compute import ComputeTarget
cpu_cluster = ComputeTarget(ws, "elkalkul")

Créer un autre Sklearn estimator qui utilise la compute nouvellement créé

In [23]:
estimator2 = SKLearn(source_directory=".", 
                    compute_target=cpu_cluster,
                    entry_script='train.py'
                   )

Créer une nouvelle experience et la submit à l'aide de l'estimator

In [24]:
exp.submit(estimator2)

Experiment,Id,Type,Status,Details Page,Docs Page
train_estimator,train_estimator_1595583234_d3002051,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


Vérifier que ça marche et debug si besoin